In [33]:
import pandas as pd
import h5py
import logging
from os import path
from pandas import HDFStore
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import EarlyStopping



# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
%reload_ext tensorboard

In [34]:
#Early stop configuration
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.0001,
  patience=2)

training_earlystop_callback = EarlyStopping(
  monitor='accuracy', min_delta=0.0001,
  patience=5)

In [35]:
#Load data from hdf5 file
root_path = "Z:/elekin"
rdo_root_path = path.join(root_path,"02-RESULTADOS/03-HANDWRITTING")
h5file = path.join(rdo_root_path, "00-OUTPUT/archimedean-")
h5filename = h5file + str(17) + ".h5"
hdf = HDFStore(h5filename)
#Load and scale timeseries between 0 and 1
raw_df = hdf['results/residues/rd'].T
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = pd.DataFrame(scaler.fit_transform(raw_df))
scaled_df['target'] = hdf.get('source/labels').values
# split data into train and test datasets (0.8, 0.2)
shuffle_df = scaled_df.sample(frac=1).reset_index(drop=True)
msk = np.random.rand(len(shuffle_df)) < 0.8
x_train = shuffle_df[msk].iloc[:,0:4096].values.astype('float32')
y_train = shuffle_df[msk]['target'].values.astype('int8')
x_test = shuffle_df[~msk].iloc[:,0:4096].values.astype('float32')
y_test = shuffle_df[~msk]['target'].values.astype('int8')

In [38]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5]))


METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_CONV_KERNEL_SIZE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [37]:
def train_test_model(hparams, num_epochs):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),# expand the dimension form (50, 4096) to (50, 4096, 1)
                      input_shape=[4096]),
     tf.keras.layers.Conv1D(filters=16, kernel_size=hparams[HP_CONV_KERNEL_SIZE],
                      strides=1, padding="causal",
                      activation=tf.nn.relu,
                      input_shape=[4096, 1]),
    tf.keras.layers.Bidirectional( tf.keras.layers.LSTM(hparams[HP_NUM_UNITS], activation=tf.nn.tanh)),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='binary_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=num_epochs, callbacks=[training_earlystop_callback]) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [39]:
def run(run_dir, hparams, num_epochs=20):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams, num_epochs)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
        for kernelsize in HP_CONV_KERNEL_SIZE.domain.values:
              hparams = {
                  HP_NUM_UNITS: num_units,
                  HP_DROPOUT: dropout_rate,
                  HP_OPTIMIZER: optimizer,
                  HP_CONV_KERNEL_SIZE: kernelsize
              }
              run_name = "run-%d" % session_num
              print('--- Starting trial: %s' % run_name)
              print({h.name: hparams[h] for h in hparams})
              run('logs-1/hparam_tuning/' + run_name, hparams, 100)
              session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam', 'conv_kernel_size': 3}
Train on 41 samples
Epoch 1/100
41/41 [==============================] - 14s 351ms/sample - loss: 0.6902 - accuracy: 0.5366
Epoch 2/100
41/41 [==============================] - 8s 191ms/sample - loss: 0.6911 - accuracy: 0.5366
Epoch 3/100
41/41 [==============================] - 7s 180ms/sample - loss: 0.6937 - accuracy: 0.5366
Epoch 4/100
41/41 [==============================] - 8s 192ms/sample - loss: 0.6888 - accuracy: 0.5366
Epoch 5/100
41/41 [==============================] - 7s 180ms/sample - loss: 0.6896 - accuracy: 0.5366
Epoch 6/100
9/1 [==============================================================================================================================================================================================================================================================================] - 1s 148ms/sample - loss: 0.6850 - accuracy: 0.5556
--- Starting trial:

In [20]:
%tensorboard --logdir logs-1/hparam_tuning

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 31876.